In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-25 15:57:18--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-25 15:57:18 (20.5 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [1]:
import minsearch
import json

In [2]:
#Download from:https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [6]:
q = 'the course has already started, can I still enroll?'

In [7]:
index.fit(documents)

In [8]:
from openai import OpenAI

In [9]:
client = OpenAI()

In [10]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"It depends on the course and the institution offering it. Many institutions do allow late enrollment, especially within the first week or two of the course. Here are a few steps you can take:\n\n1. **Check the Course Policy:** Review the course syllabus or the institution's academic calendar to understand their policy on late enrollment.\n\n2. **Contact the Instructor or Administration:** Reach out to the course instructor or the academic office to explain your situation and see if late enrollment is an option.\n\n3. **Understand the Catch-Up Requirements:** If late enrollment is permitted, ask about any materials you need to catch up on and if there are any initial assignments or tests you’ve missed.\n\n4. **Complete Necessary Paperwork:** There might be additional forms or steps you need to complete for late enrollment. Make sure you follow all administrative procedures.\n\n5. **Evaluate Your Ability to Catch Up:** Consider realistically whether you'll be able to catch up on missed 

In [11]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [13]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [14]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [15]:
query = 'how do I run kafka?'
rag(query)

'To run Kafka, you can follow these instructions depending on the context:\n\n**Java Kafka:**\n1. Navigate to your project directory.\n2. Run the following command in the terminal:\n    ```sh\n    java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n    ```\n\n**Python Kafka:**\n1. Create a virtual environment and install the required packages:\n    ```sh\n    python -m venv env\n    source env/bin/activate\n    pip install -r ../requirements.txt\n    ```\n    For Windows, use:\n    ```sh\n    env\\Scripts\\activate\n    ```\n2. Activate the virtual environment:\n    ```sh\n    source env/bin/activate\n    ```\n3. Deactivate the virtual environment when done:\n    ```sh\n    deactivate\n    ```\n   \n4. Ensure Docker images are up and running before running your Python Kafka scripts.\n\nIf there is a permissions issue with a script (like `build.sh`):\n1. Navigate to the directory containing the script.\n2. Run the command to make the script e

In [16]:
from elasticsearch import Elasticsearch

In [17]:
es_client = Elasticsearch('http://localhost:9200') 

In [18]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [19]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [20]:
from tqdm.auto import tqdm

/Users/ppathorn/Downloads/homework1_llm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████| 948/948 [00:03<00:00, 274.85it/s]


In [25]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [23]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [24]:
query = 'I just disovered the course. Can I still join it?'
rag(query)

'Yes, you can still join the course even if you discovered it after the start date. You are eligible to submit the homework, but be mindful of the deadlines for turning in the final projects to avoid last-minute challenges.'